In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np

In [2]:
def load_json(file):
    with open(f'../data/{file}.json','r') as f:
        return json.loads(f.read().replace('\\\\','\\').encode('utf-8'))

In [3]:
events = load_json('events_England')
ev_df = pd.DataFrame(events)
tag_df = pd.read_csv('../data/tagcodes.csv')
tag_dict = dict(zip(tag_df.Tag, tag_df.Description))

In [4]:
for i in range(6):
    ev_df[f'tag{i}'] = ev_df['tags'].str[i].str['id'].map(tag_dict).str.lower()

In [5]:
players = load_json('players')
players_dict = dict(zip(pd.DataFrame(players).wyId,pd.DataFrame(players).shortName))
teams = load_json('teams')
teams_dict = dict(zip(pd.DataFrame(teams).wyId,pd.DataFrame(teams).name))
ev_df['player'] = ev_df['playerId'].map(players_dict)
ev_df['team'] = ev_df['teamId'].map(teams_dict)
ev_df['location'] = ev_df['positions'].str[0]
ev_df['target'] = ev_df['positions'].str[1]
e_df = ev_df[['matchId','eventSec','matchPeriod','team','location','target','player','eventName','subEventName','tag0','tag1','tag2','tag3','tag4','tag5']]
e_df['notes'] = e_df[['tag0','tag1','tag2','tag3','tag4','tag5']].values.tolist()
e_df['tags'] = e_df['notes'].astype(str)
e_df['accurate'] = e_df.tags.str.contains('accurate')
e_df['accurate'] = np.where(e_df.tags.str.contains('not accurate'), 0, e_df.accurate)
e_df.drop(columns=['tag0','tag1','tag2','tag3','tag4','tag5','notes'],inplace=True)

<ipython-input-5-261c910ba906>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['notes'] = e_df[['tag0','tag1','tag2','tag3','tag4','tag5']].values.tolist()
<ipython-input-5-261c910ba906>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['tags'] = e_df['notes'].astype(str)
<ipython-input-5-261c910ba906>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [6]:
play = 0
plays = []
next_action = []
match_play = []
matchId = e_df.iloc[0].matchId
for i in range(len(e_df)):
    if i>1:
        row_minus2 = row_minus1
    if i>0:
        row_minus1 = row
    row = e_df.iloc[i]
    if i == 0:
        row_minus1 = row
        row_minus2 = row
    
    if row.eventName == 'Free Kick':
        play += 1
    elif 'goal' in row_minus2.tags and 'goal' in row_minus1.tags:
        # (there are generally two events marked goal -- the shot and the attempted save)
        play +=1
    
    if row.matchId == matchId:
        pass
    else:
        play = 0
        matchId = row.matchId
    plays.append(play)
    match_play.append(f'{matchId}-{play}')

e_df['in_play'] = pd.Series(plays)
e_df['match_play'] = pd.Series(match_play)

<ipython-input-6-d913a723268f>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['in_play'] = pd.Series(plays)
<ipython-input-6-d913a723268f>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['match_play'] = pd.Series(match_play)


In [7]:
ps_sequence = np.full(len(e_df),'0000000000000000000000000000000000')
played_to = np.full(len(e_df), '                                    ')
to_team = np.full(len(e_df), '                                       ')

for match in e_df.matchId.unique():
# for match in [e_df.matchId.unique()[0]]:
    match_df = e_df.loc[e_df.matchId == match]
    for team in match_df.team.unique():
        for play in match_df.in_play.unique():
            team_play_df = match_df.loc[match_df.in_play == play].loc[match_df.team == team]
            possession_sequence = 0
            for i in range(len(team_play_df)):
                if i>0:
                    previous_row = row
                
                row = team_play_df.iloc[i]
                
               
                if i > 0:
                    if row.location != target:
                        possession_sequence += 1
                        played_to[previous_row.name] = 'Opposition'
                        to_team[previous_row.name] = 'Opposition'
                    else:
                        played_to[previous_row.name] = row.player
                        to_team[previous_row.name] = row.team
                team_name = team.split(' ')
                team_abbr = ''
                for word in team_name:
                    team_abbr = team_abbr + word[:3]
                ps_sequence[row.name] = f'{match}-{play}-{team_abbr}-{possession_sequence}'
                target = row.target
            
e_df['possession'] = ps_sequence            
e_df['played_to'] = played_to
e_df['to_team'] = to_team

<ipython-input-7-e79c0d56b6b1>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['possession'] = ps_sequence
<ipython-input-7-e79c0d56b6b1>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['played_to'] = played_to
<ipython-input-7-e79c0d56b6b1>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [8]:
# e_df['played_to'] = np.where(e_df['played_to'] == '                                    ', 'Opposition', e_df['played_to'])

In [9]:
e_df['to_team'] = np.where(e_df['to_team'] == '                                       ', 'Opposition', e_df['to_team'])

<ipython-input-9-434ce155762d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['to_team'] = np.where(e_df['to_team'] == '                                       ', 'Opposition', e_df['to_team'])


In [10]:
for foo in ('location','target'):
    for goo in ('x','y'):
        e_df[f'{foo}_{goo}'] = e_df[foo].str[goo]

<ipython-input-10-2d6eab7497c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df[f'{foo}_{goo}'] = e_df[foo].str[goo]


In [11]:
# change target coordinates of goal to inside goal
e_df['target_x'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),
                            100, e_df.target_x )
e_df['target_y'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),
                            50, e_df.target_y )

<ipython-input-11-1d704e52991d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['target_x'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),
<ipython-input-11-1d704e52991d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['target_y'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),


In [12]:
e_df['subEventName'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),
                            'Goal', e_df.subEventName )

<ipython-input-12-ae24da1ccb5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['subEventName'] = np.where((e_df.tags.str.contains("'goal'")) & (e_df.eventName != 'Save attempt'),


In [13]:
e_df['eventName'] = np.where(e_df['eventName'] == 'Others on the ball', 'On the Ball', e_df['eventName'])

<ipython-input-13-721fb6b2b6c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['eventName'] = np.where(e_df['eventName'] == 'Others on the ball', 'On the Ball', e_df['eventName'])


In [14]:
e_df['eventName'] = np.where(e_df['eventName'] == 'Duel', 'Challenge', e_df['eventName'])

<ipython-input-14-5e7bf0d6f265>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['eventName'] = np.where(e_df['eventName'] == 'Duel', 'Challenge', e_df['eventName'])


In [15]:
e_df['played_to'] = np.where(e_df.subEventName == 'Goal', 'Goal', e_df.played_to)
e_df['to_team'] = np.where(e_df.subEventName == 'Goal', 'Goal', e_df.to_team)

<ipython-input-15-e301300ce245>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['played_to'] = np.where(e_df.subEventName == 'Goal', 'Goal', e_df.played_to)
<ipython-input-15-e301300ce245>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['to_team'] = np.where(e_df.subEventName == 'Goal', 'Goal', e_df.to_team)


In [16]:
# change location coordinates of goal kick
e_df['location_x'] = np.where((e_df.subEventName == 'Goal kick'), 5, e_df.location_x)
e_df['location_y'] = np.where((e_df.subEventName == 'Goal kick'), 50, e_df.location_y)

<ipython-input-16-1578e4c71fa2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['location_x'] = np.where((e_df.subEventName == 'Goal kick'), 5, e_df.location_x)
<ipython-input-16-1578e4c71fa2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['location_y'] = np.where((e_df.subEventName == 'Goal kick'), 50, e_df.location_y)


In [17]:
# add time in minutes and seconds
e_df['minute'] = e_df.eventSec//60
e_df.minute = e_df.minute.astype(int)
e_df['seconds'] = e_df.eventSec % 60
e_df.seconds = e_df.seconds.astype(int)
e_df['time'] = e_df.minute.astype(str).str.zfill(2) + ':' + e_df.seconds.astype(str).str.zfill(2)
e_df.drop(columns=['minute','seconds'],inplace=True)

<ipython-input-17-cc377ac826a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['minute'] = e_df.eventSec//60
/home/peterprescott/miniconda3/envs/babel/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [18]:
e_df.drop(columns=['location','target'], inplace=True)

In [19]:
e_df.rename(columns={'player':'source','played_to':'target'},inplace=True)

In [20]:
matches = load_json('matches_England')

In [21]:
match_df = pd.DataFrame(matches)

In [22]:
match_lineups = {}

for match in match_df.wyId.unique():
    match_lineups[match] = {}
    for team in match_df.loc[match_df.wyId == match].teamsData.values[0].keys():
        match_lineups[match][teams_dict[int(team)]] = list(pd.Series(pd.DataFrame(match_df.loc[
            match_df.wyId == match].teamsData.str[str(team)].str['formation'].str['lineup'].values[0]).playerId.values).map(players_dict))

In [23]:
def firstXI(row):
    return row.source in match_lineups[row.matchId][row.team]

In [24]:
e_df['FirstXI'] = e_df.apply(firstXI, axis=1)

In [25]:
h_a_map = {}
for match in match_df.wyId.unique():
    h_a_map[match] = {}
    for team in match_df.loc[match_df.wyId == match].teamsData.values[0].keys():
        status = match_df.loc[match_df.wyId==match].teamsData.str[str(team)].str['side'].values[0].capitalize()
        h_a_map[match][team] = status

In [26]:
def home_or_away(row):
    return h_a_map[row.matchId][str(row.teamId)]

In [27]:
e_df['home_or_away'] = ev_df.apply(home_or_away, axis=1)

In [28]:
e_df['to_team'] = np.where(e_df.to_team == e_df.team, e_df.home_or_away, e_df.to_team)

opposition = {'Home':'Away','Away':'Home'}

e_df['to_team'] = np.where(e_df.to_team == 'Opposition', e_df.home_or_away.map(opposition), e_df.to_team)

e_df['to_team'] = np.where(e_df.tags.str.contains('own goal'), 'Own Goal', e_df.to_team)

whose_goal = {'Home':'Home Goal', 'Away':'Away Goal'}
e_df['to_team'] = np.where(e_df.to_team == 'Goal', e_df.home_or_away.map(whose_goal), e_df.to_team)

whose_own_goal = {'Home':'Away Goal', 'Away':'Home Goal'}
e_df['to_team'] = np.where(e_df.to_team == 'Own Goal', e_df.home_or_away.map(whose_own_goal), e_df.to_team)

In [29]:
e_df.drop(columns=['tags','in_play'],inplace=True)

In [30]:
e_df.to_csv('../data/processed_events_England.csv')

some probabilistic notions regarding random walks on graphs can be stated in terms of voltage,
flow of current, effective resistance, etc., and then electric laws can be applied in order to get simple
electric solutions to more or less elaborate probabilistic problems.